In [1]:
import otagrum

import openturns as ot

import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

In [2]:
bn = gum.BayesNet.fastPrototype('A->B->C;D->E')
bn

(gum::BayesNet<double>@0x555a7ce80190) BN{nodes: 5, arcs: 3, domainSize: 32, parameters: 16, compression ratio: 50% }

In [3]:


ndag=otagrum.NamedDAG(bn)
order=ndag.getTopologicalOrder()

jointDistributions=[]
for i in range(order.getSize()):
    d = 1 + ndag.getParents(i).getSize()
    R = ot.CorrelationMatrix(d)
    for i in range(d):
        for j in range(i):
            R[i, j] = 0.8
    jointDistributions.append(ot.Student(5.0, [0.0]*d, [1.0]*d, R).getCopula())

copula = otagrum.ContinuousBayesianNetwork(ndag, jointDistributions)
sample = copula.getSample(5000)
learner=otagrum.ContinuousPC(sample)
g=learner.inferSkeleton()
print(learner.inferPDAG(g))
gnb.sideBySide(bn,gnb.getDot(g.toDot()),gnb.getDot(learner.learnDAG().toDot()))

{0,1,2,3,4} , {} , {1--4,2--4,3--4}


G <!-- A --> A A <!-- B --> B B <!-- A->B --> A->B <!-- C --> C C <!-- B->C --> B->C <!-- D --> D D <!-- E --> E E <!-- D->E --> D->E,no_name <!-- 0 --> 0 0 <!-- 1 --> 1 1 <!-- 4 --> 4 4 <!-- 1->4 --> 1->4 <!-- 2 --> 2 2 <!-- 2->4 --> 2->4 <!-- 3 --> 3 3 <!-- 3->4 --> 3->4,G <!-- B --> B B <!-- E --> E E <!-- B->E --> B->E <!-- D --> D D <!-- E->D --> E->D <!-- C --> C C <!-- E->C --> E->C


In [4]:
def test(x,y):
    b=learner.getPValue(x,y)
    c=learner.getTTest(x,y)
    if learner.isRemoved(x,y):
        a=learner.getSepsetNames(x,y)
        print(f"{x}-{y} : p={b} ({c}) cut by {a}")
    else:
        print(f"{x}-{y} : p={b} ({c})")
        
test("A","B")
test("B","C")
test("B","E")

A-B : p=0.9298945181043068 (-0.0879775516886975) cut by ()
B-C : p=0.8303825568296407 (79.67667097415905) cut by ('E',)
B-E : p=0.0 (78.02702245341769)


In [4]:
learner.getRemoved()

<Swig Object of type 'std::vector< gum::Edge,std::allocator< gum::Edge > > *' at 0x7ff766ba2b40>